# Knowledge Structure Graph Generator

## 1. DB연결

In [226]:
import pymysql

In [227]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [228]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [229]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [230]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
  DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0     감초      scs  리코리스,미국,2.76\n약용,식품,3.21\n하루,이상,3.9\n이상,단맛,3.9...
1     대마      scs  마약,법률,3.06\n약물,중독,3.26\n합법,국가,3.38\n한국,미국,3.87...
2      차      scs  당나라,시기,4.32\n우려,식물,4.81\n재배,지방,4.97\n지방,식물,4.9...
3      콩      scs  두부,된장,1.8\n고기,짜파게티,2.53\n여성,에스트로겐,3.0\n요리,미국,3...
4     황기      scs  독성,약재,1.0\n이상,한약재,1.0\n이상,의약품,1.0\n한약재,의약품,1.0...


In [231]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [391]:
file_name = "황기" 

#### 추출하기 원하는 인덱스 선정

In [392]:
data = knowledge_structure[4] #인덱스만 변경하기
print(data)

독성,약재,1.0
이상,한약재,1.0
이상,의약품,1.0
한약재,의약품,1.0
비위,설사,1.63
부작용,낫다,2.76
약효,한약,3.54
이상,이유,3.54
이유,한약재,3.54
이유,요리,3.54
이유,의약품,3.54
삼계탕,주재료,4.0
삼계탕,한약,4.0
비위,부작용,5.1
삼계탕,이유,5.27



In [393]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [394]:
#노드 추가
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    print(value)

['독성', '약재', '1.0']
['이상', '한약재', '1.0']
['이상', '의약품', '1.0']
['한약재', '의약품', '1.0']
['비위', '설사', '1.63']
['부작용', '낫다', '2.76']
['약효', '한약', '3.54']
['이상', '이유', '3.54']
['이유', '한약재', '3.54']
['이유', '요리', '3.54']
['이유', '의약품', '3.54']
['삼계탕', '주재료', '4.0']
['삼계탕', '한약', '4.0']
['비위', '부작용', '5.1']
['삼계탕', '이유', '5.27']


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [433]:
import networkx as nx

In [434]:
G = nx.Graph()

In [435]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [436]:
degree_list = [degree for degree in G.degree()] # 각 노드별 차수 저장

In [437]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    tmp['degree'] = degree_list[num][1]

    
    # 색상은 파랑색으로 지정
    tmp['color'] = "#97C2FC"
    
#     # 차수가 3이상이면 빨간색 지정
#     if(degree_list[num][1] >= 3):
#         tmp['color'] = "#FB7E81"
#     else:
#         tmp['color'] = "#97C2FC"
        
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [438]:
nodes_list

[{'id': 0, 'label': '독성', 'degree': 1, 'color': '#97C2FC'},
 {'id': 1, 'label': '약재', 'degree': 1, 'color': '#97C2FC'},
 {'id': 2, 'label': '이상', 'degree': 3, 'color': '#97C2FC'},
 {'id': 3, 'label': '한약재', 'degree': 3, 'color': '#97C2FC'},
 {'id': 4, 'label': '의약품', 'degree': 3, 'color': '#97C2FC'},
 {'id': 5, 'label': '비위', 'degree': 2, 'color': '#97C2FC'},
 {'id': 6, 'label': '설사', 'degree': 1, 'color': '#97C2FC'},
 {'id': 7, 'label': '부작용', 'degree': 2, 'color': '#97C2FC'},
 {'id': 8, 'label': '낫다', 'degree': 1, 'color': '#97C2FC'},
 {'id': 9, 'label': '약효', 'degree': 1, 'color': '#97C2FC'},
 {'id': 10, 'label': '한약', 'degree': 2, 'color': '#97C2FC'},
 {'id': 11, 'label': '이유', 'degree': 5, 'color': '#97C2FC'},
 {'id': 12, 'label': '요리', 'degree': 1, 'color': '#97C2FC'},
 {'id': 13, 'label': '삼계탕', 'degree': 3, 'color': '#97C2FC'},
 {'id': 14, 'label': '주재료', 'degree': 1, 'color': '#97C2FC'}]

In [439]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '독성', 'degree': 1, 'color': '#97C2FC'}, {'id': 1, 'label': '약재', 'degree': 1, 'color': '#97C2FC'}, {'id': 2, 'label': '이상', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '한약재', 'degree': 3, 'color': '#97C2FC'}, {'id': 4, 'label': '의약품', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '비위', 'degree': 2, 'color': '#97C2FC'}, {'id': 6, 'label': '설사', 'degree': 1, 'color': '#97C2FC'}, {'id': 7, 'label': '부작용', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '낫다', 'degree': 1, 'color': '#97C2FC'}, {'id': 9, 'label': '약효', 'degree': 1, 'color': '#97C2FC'}, {'id': 10, 'label': '한약', 'degree': 2, 'color': '#97C2FC'}, {'id': 11, 'label': '이유', 'degree': 5, 'color': '#97C2FC'}, {'id': 12, 'label': '요리', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '삼계탕', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '주재료', 'degree': 1, 'color': '#97C2FC'}];


In [440]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'독성': 0, '약재': 1, '이상': 2, '한약재': 3, '의약품': 4, '비위': 5, '설사': 6, '부작용': 7, '낫다': 8, '약효': 9, '한약': 10, '이유': 11, '요리': 12, '삼계탕': 13, '주재료': 14}


In [441]:
edges_list = []

edge_max = 0.0
edge_min = 7.0

for edge in G.edges(data='weight'):
#     print("step: " + edge[2])
    tmp = float(edge[2])

    #최댓값 저장
    if(edge_max <= tmp):
        edge_max = tmp
    
    #최솟값 저장
    if(edge_min >= tmp):
        edge_min = tmp
        
#     print("edge_max: " + str(edge_max))
#     print("edge_min: " + str(edge_min))
#     print()

In [442]:
# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']   = reverse_graph.get(edge[0])     # 'hemp'검색시 인덱스값
    tmp['to']     = reverse_graph.get(edge[1])     # 'cannabi'검색시 인덱스값
    tmp['label']  = edge[2]                        # 가중치 저장
    
    length = 7.0 - float(edge[2])
    convert_edge_min = 7.0 - edge_max
    minmax_scale = ((length - convert_edge_min) / (edge_max - edge_min)) * 500        # 길이를 min max 스케일링
    tmp['value']  = round(minmax_scale, 2)         # Edge 두께를 가중치의 크기에 따라 변경(숫자가 클수록 두껍게)
    
    length = float(edge[2])
    minmax_scale = ((length - edge_min)/(edge_max - edge_min)) * 500
    tmp['length'] = round(minmax_scale +220, 2)    # Edge 길이를 min max 스케일링
    
    tmp['color'] = "#d3d3d3"                      

    edges_list.append(tmp)

In [443]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '1.0', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 2, 'to': 3, 'label': '1.0', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 2, 'to': 4, 'label': '1.0', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 2, 'to': 11, 'label': '3.54', 'value': 202.58, 'length': 517.42, 'color': '#d3d3d3'}, {'from': 3, 'to': 4, 'label': '1.0', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 3, 'to': 11, 'label': '3.54', 'value': 202.58, 'length': 517.42, 'color': '#d3d3d3'}, {'from': 4, 'to': 11, 'label': '3.54', 'value': 202.58, 'length': 517.42, 'color': '#d3d3d3'}, {'from': 5, 'to': 6, 'label': '1.63', 'value': 426.23, 'length': 293.77, 'color': '#d3d3d3'}, {'from': 5, 'to': 7, 'label': '5.1', 'value': 19.91, 'length': 700.09, 'color': '#d3d3d3'}, {'from': 7, 'to': 8, 'label': '2.76', 'value': 293.91, 'length': 426.09, 'color': '#d3d3d3'}, {'from': 9, 'to': 10, 'label': '3.54', 'value': 202.58, 'l

In [444]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [445]:
print(graph_data)

var nodes = [{'id': 0, 'label': '독성', 'degree': 1, 'color': '#97C2FC'}, {'id': 1, 'label': '약재', 'degree': 1, 'color': '#97C2FC'}, {'id': 2, 'label': '이상', 'degree': 3, 'color': '#97C2FC'}, {'id': 3, 'label': '한약재', 'degree': 3, 'color': '#97C2FC'}, {'id': 4, 'label': '의약품', 'degree': 3, 'color': '#97C2FC'}, {'id': 5, 'label': '비위', 'degree': 2, 'color': '#97C2FC'}, {'id': 6, 'label': '설사', 'degree': 1, 'color': '#97C2FC'}, {'id': 7, 'label': '부작용', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '낫다', 'degree': 1, 'color': '#97C2FC'}, {'id': 9, 'label': '약효', 'degree': 1, 'color': '#97C2FC'}, {'id': 10, 'label': '한약', 'degree': 2, 'color': '#97C2FC'}, {'id': 11, 'label': '이유', 'degree': 5, 'color': '#97C2FC'}, {'id': 12, 'label': '요리', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '삼계탕', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '주재료', 'degree': 1, 'color': '#97C2FC'}];

var edges = [{'from': 0, 'to': 1, 'label': '1.0', 'value': 500.0, 'length': 220.0, 'color'

In [446]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

황기.js 파일 생성 완료


### 2. keyword.html

In [409]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [410]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>
    """

In [411]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
        {width: 100%;height: 1000px;border: 1px solid lightgray;}
    </style>
</head>
<body>
    """

In [412]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [413]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [414]:
option = """
        var options = 
        { "nodes": {"shape" : "circle",
                    "font":{"size": 35}},
        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, 
            "smooth": {"type":"continuous","roundness": 0.5}},
            "physics": {"barnesHut": {"centralGravity": 0.05},
            "minVelocity": 0.75}};
        """

In [415]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [416]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [417]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script>\n    <script src="../../results/result_webgraph/data/황기.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n        {width: 100%;height: 1000px;border: 1px solid lightgray;}\n    </style>\n</head>\n<body>\n    <p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "nodes": {"shape" : "circle",\n                    "font":{"size": 35}},\n        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, \n            "smooth": {"type":"continuous","roundness": 0.5}},\n            "physics": {"barnesHut": {"centralGravity": 0.05},\n            "minVelocity": 0.75}};\n        var network = new vis.Network(container, data, opti

In [418]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

황기.html 파일 생성 완료
